In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as stats
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

import sys, random

%load_ext autoreload
%autoreload 2

sys.path.append('..')
from util import provas,load_padr,score_inep, notas_sisu, params_inep

# 1. Escolhe uma prova e um candidato

In [3]:
areawidg = widgets.Dropdown(
    options=['CN','CH','MT'],
    value='CN',
    description='Area:',
)
anowidg = widgets.Dropdown(
    options=range(2009,2023),
    value=2022,
    description='Ano:',
)

notawdg = widgets.IntRangeSlider(
    value=[300, 1000],
    min=300,
    max=1000,
    step=20,
    description='Notas min/max:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)


def random_candidates(ano=2022,area='CH',N=1,notaminmax=[300,1000]):
    'return a random candidate, their answers and their score given by INEP'

    global prova, padr, itens, params, itemwdg,itemwdg_elim
    prova = random.choice(provas(ano,area))
    params = params_inep(prova)

    itens_a = pd.qcut(params['a_inep'],3,labels=['Baixa','Média','Alta'])
    itens_b = pd.qcut(params['b_inep'],3,labels=['Baixa','Média','Alta'])
    itens = pd.concat([itens_a,itens_b],axis=1)
    item_options = [(f"Discriminação {row['a_inep']}, Difficuldade {row['b_inep']}",index) for index, row in itens.iterrows()]
    item_options.insert(0,('',''))
    itemwdg = widgets.Dropdown(
        placeholder='Escolhe um item',
        options=item_options,
        description='Flip:',
        ensure_option=False,
        disabled=False,
        )
    itemwdg_elim = widgets.Dropdown(
        placeholder='Escolhe um item',
        options=item_options,
        description='Cancelar:',
        ensure_option=False,
        disabled=False,
        )
    
    padr, nota_inep = load_padr(prova,3000,True)
    notamin,notamax = notaminmax[0],notaminmax[1]
    
    nota_inep = nota_inep.query("nota_inep > @notamin and nota_inep < @notamax")
    cands = nota_inep.sample(N)
    padr = cands.join(padr)
    nota = padr['nota_inep'].iloc[0]
    print(f"Prova: {prova}, nota segundo o INEP: {nota}")

    #display(padr)
    return


interact(random_candidates,area=areawidg,ano=anowidg,notaminmax=notawdg);

#padr, prova = random_candidates(2022,'CN',5)

interactive(children=(Dropdown(description='Ano:', index=13, options=(2009, 2010, 2011, 2012, 2013, 2014, 2015…

In [4]:
def flip_item(padr,item):
    padr2 = padr.copy()
    padr2[item] = 1 - padr[item] # flip the answers of one item
    return padr2

def elim_item(padr,item):
    padr2 = padr.copy()
    return padr2.drop(columns=[item])


In [5]:
def notaenem(flip='',elim='',method='EAP'):
    padr1 = padr.copy()
    params1 = params.copy()
    if flip:
        padr1 = flip_item(padr1,flip)
    if elim:
        padr1 = elim_item(padr1,elim)
        params1 = params_inep(prova).drop(elim).copy()
    
    nota1 = score_inep(padr1.iloc[:,3:],prova=prova,params=params1,method=method,enemscale=True)

    nota = padr.iloc[0,2]
    notanova = nota1.iloc[0,0]
    se = nota1.iloc[0,1]
    diff = notanova - nota
    print(f"Nota dado pelo INEP: {nota}")
    print(f"\nNota nova: {notanova:.1f} ± {se:.1f}")
    print(f"Diferença: {diff:.1f}")
    
    cursos = notas_sisu(2024,1)
    cursosok = cursos.query("@notanova >= notacorte")
    cursosnok = cursos.query("@notanova < notacorte")

    print(f"Cursos que pode cursar: {len(cursosok)} (de um total de {len(cursos)} cursos)")
    print(f"Cursos que não pode cursar: {len(cursosnok)} (de um total de {len(cursos)} cursos)")

    print("Alguns cursos que pode cursar:")
    display(cursosok.sample(10))

    
metwdg = widgets.Dropdown(
    options=['EAP','MAP','ML'],
    value='EAP',
    description='Método:'
)

interact(notaenem,flip=itemwdg,elim=itemwdg_elim,method = metwdg);

interactive(children=(Dropdown(description='Flip:', options=(('', ''), ('Discriminação Média, Difficuldade Méd…